# Implementación de un Sistema de Recomendación de películas: MovieLens


* En este Notebook vamos a implementar y evaluar un ***Sistema de Recomendación basado en Filtrado Colaborativo con KNN*** usando la base de datos de [MovieLens 100K](https://grouplens.org/datasets/movielens/).


* Esta base de datos contiene 100.000 votos con puntuaciones de 1 a 5 de 943 usuarios sobre 1682 películas.


* Este dataset ha sido dividido en votaciones de entrenamiento (75%) y votaciones de test (25%).


* Para ***implementar y evaluar este Sistema de Recomendación*** realizaremos los siguientes paso:
<span></span><br>
    1. [Paquetes requeridos](#M1)
<span></span><br>
    2. [Análisi exploratorio (EDA)](#M2)
<span></span><br>
    3. [Limpieza de datos](#M3)
<span></span><br>
    4. [Separar los datos en entrenamiento y prueba](#M4)
<span></span><br>
    5. [Definir los parámetros de similitud para los modelos](#M5)
<span></span><br>
    6. [Entrenar modelos KNN](#M6)
<span></span><br>
    7. [Cálculo de las predicciones](#M7)
<span></span><br>
    8. [Cálculo del MAE(Mean Absolute Error) y RMSE(Root Mean Squared Error)](#M8)
<span></span><br>
    9. [Visualización del MAE y el RMSE de cada modelo](#M9)



<hr>

# <a name="M1">1. Paquetes requeridos

In [20]:
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

<hr>

# <a name="M2">2. Análisi exploratorio (EDA)

* Cargamos el dataset Movilens 100k.

In [21]:
# Estructura (usuario, ítem, rating, timestamp)
data = Dataset.load_builtin('ml-100k')

* Mostramos los datos.

In [22]:
# Convertimos el conjunto de datos en un dataframe
df = pd.DataFrame(data.raw_ratings, columns=['user_id', 'movie_id', 'rating', 'timestamp'])
df.head()

,user_id,movie_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [23]:
# Mostramos las últimas cinco filas del dataframe
df.tail()

,user_id,movie_id,rating,timestamp
99995,880,476,3.0,880175444
99996,716,204,5.0,879795543
99997,276,1090,1.0,874795795
99998,13,225,2.0,882399156
99999,12,203,3.0,879959583


* Visualizamos la descripción total de todos los datos del dataframe.

In [24]:
# El método describe() de Pandas es una función muy útil que se utiliza para obtener estadísticas descriptivas de un DataFrame o de una Serie.
df.describe(include='all')

,user_id,movie_id,rating,timestamp
count,100000,100000,100000.000000,100000
unique,943,1682,NaN,49282
top,405,50,NaN,891033606
freq,737,583,NaN,12
mean,NaN,NaN,3.529860,NaN
std,NaN,NaN,1.125674,NaN
min,NaN,NaN,1.000000,NaN
25%,NaN,NaN,3.000000,NaN
50%,NaN,NaN,4.000000,NaN
75%,NaN,NaN,4.000000,NaN


* Convertimos los datos no númericos a numéricos.

In [25]:
df['user_id'] = pd.to_numeric(df['user_id'])
df['movie_id'] = pd.to_numeric(df['movie_id'])

* Mostramos los datos con la conversión realizada.

In [26]:
df.describe(include='all')

,user_id,movie_id,rating,timestamp
count,100000.00000,100000.000000,100000.000000,100000
unique,NaN,NaN,NaN,49282
top,NaN,NaN,NaN,891033606
freq,NaN,NaN,NaN,12
mean,462.48475,425.530130,3.529860,NaN
std,266.61442,330.798356,1.125674,NaN
min,1.00000,1.000000,1.000000,NaN
25%,254.00000,175.000000,3.000000,NaN
50%,447.00000,322.000000,4.000000,NaN
75%,682.00000,631.000000,4.000000,NaN


<hr>

# <a name="M3">3. Limpieza de datos

* Verificamos si hay valores nulos.

In [27]:
# Verifica si existen columnas con datos nulos
for col in df.columns:
    print(col,' has nulls =>', df['user_id'].isnull().any())

user_id  has nulls => False
movie_id  has nulls => False
rating  has nulls => False
timestamp  has nulls => False


* Eliminamos los valores nulos si existiesen.

In [28]:
# Elimina valores nulos si existen
null_columns = df.columns[df.isnull().any()]
if null_columns.size > 0:
    print('Eliminando valores nulos...', end=' ')
    df.dropna(inplace=True)
    print('Valores nulos eliminados')
else:
    print('No hay valores nulos')

No hay valores nulos


* Verificamos si hay filas duplicadas.

In [29]:
#Verifica si existen filas duplicadas
are_duplicates = df.duplicated()
print(are_duplicates)

0        False
1        False
2        False
3        False
4        False
         ...  
99995    False
99996    False
99997    False
99998    False
99999    False
Length: 100000, dtype: bool


* Eliminamos los valores duplicados si existiesen.

In [30]:
if True in are_duplicates.values:
    print('Eliminando duplicados...', end=' ')
    df.drop_duplicates(inplace=True)
    print('Valores duplicados eliminados')
else:
    print('No hay duplicados')

No hay duplicados


<hr>

# <a name="M4">4. Separar los datos en entrenamiento y prueba

In [31]:
trainset, testset = train_test_split(data, test_size=0.25,random_state=33)

<hr>

# <a name="M5">5. Definir los parámetros de similitud para los modelos

In [32]:
# Inicializamos un diccionario vacío llamado options. Este diccionario contendrá todas las configuraciones de las diferentes métricas de similitud.
options = {}

# Hacemos este bucle porque se crea un diccionario optx para cada tipo de similitud y luego es almacenado en la variable opt
for opt in ['cosine', 'pearson', 'msd', 'pearson_baseline']:
    optx = {
        #cosine, pearson, msd, pearson-baseline
        'name': opt, 
        # True si la similitud es entre usuarios, False si es entre ítems 
        'user_based': True
    }
    # Actualiza el valor del diccionario options con todas las simulitudes calculadas una por una de la variable opt
    options.update({opt:optx})

# Intancias del algoritmo KNN con diferentes configuraciones de similitud:
knn_cosine = KNNBasic(k=40, min_k=10, sim_options=options.get('cosine'))
knn_pearson = KNNBasic(k=40, min_k=10, sim_options=options.get('pearson'))
knn_msd = KNNBasic(k=40, min_k=10, sim_options=options.get('msd'))
knn_pearson_baseline = KNNBasic(k=40, min_k=10, sim_options=options.get('pearson_baseline'))

<hr>

# <a name="M6">6. Entrenar modelos KNN

In [33]:
knn_cosine.fit(trainset)
knn_pearson.fit(trainset)
knn_msd.fit(trainset)
knn_pearson_baseline.fit(trainset)

knn_models = {'cosine':knn_cosine, 'pearson':knn_pearson, 'msd':knn_msd, 'pearson_baseline':knn_pearson_baseline}

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


<hr>

# <a name="M7">7. Cálculo de las predicciones

In [34]:
predictions_cosine = knn_cosine.test(testset)
predictions_pearson = knn_pearson.test(testset)
predictions_msd = knn_msd.test(testset)
predictions_pearson_baseline = knn_pearson_baseline.test(testset)

<hr>

# <a name="M8">8. Cálculo del MAE (Mean Absolute Error) y RMSE (Root Mean Squared Error)

In [35]:
rmses = []
maes = []
for prediction in [predictions_cosine, predictions_pearson, predictions_msd, predictions_pearson_baseline]:
    rmse = accuracy.rmse(prediction, verbose=False)
    mae = accuracy.mae(prediction, verbose=False)
    rmses.append(rmse)
    maes.append(mae)

<hr>

# <a name="M9">9. Visualización del MAE y el RMSE de cada modelo

In [36]:
df_result = pd.DataFrame(data=[(rmse, mae) for rmse,mae in zip(rmses, maes)], 
columns=['RMSE','MAE'], index=['cosine','pearson','msd','pearson_baseline'])

df_result

,RMSE,MAE
cosine,1.018389,0.806459
pearson,1.015303,0.807778
msd,0.983713,0.778126
pearson_baseline,1.010000,0.802104


<code> Como podemos observar en los diferentes modelos calculados, el que tiene un MAE y un RMSE bajos, es el modelo en donde se aplica el MSD. Esto indica que es un modelo eficiente y que puede ser utilizado con datos reales donde se requieren predicciones precisas. </code>